In [111]:
import os
import sys
import re
import pandas as pd
import zipfile
from enum import Enum, EnumMeta
from io import StringIO
from Tables import Tables
from Spells import Spells
from Items import Items
from IPython.display import display

In [112]:
class BaseOps:
    
    @staticmethod
    def bytes2Num(barray, signed=False, order='little'):
        return int.from_bytes(barray, signed=signed, byteorder=order)
    
    @staticmethod
    def num2Bytes(num, length, signed=False, order='little'):
        return num.to_bytes(length, signed=signed, byteorder=order)
    

class DataPiece(BaseOps):
    
    def __init__(self, position, length):
        self.position = position
        self.length = length
        self.end = self.position + self.length
        self.signed = False
        self.order = 'little'
    
    def getBytes(self, buffer, seg_offset):
        return buffer[seg_offset + self.position:
                      seg_offset + self.end]
    
    def getValue(self, buffer, seg_offset):
        return self.bytes2Num(buffer[seg_offset + self.position:
                                     seg_offset + self.end],
                              self.signed,
                              self.order)
    
    def setValue(self, buffer, seg_offset, new_value):
        if not isinstance(new_value, int):
            new_value = new_value.value
        buffer[seg_offset + self.position: 
               seg_offset + self.end] = self.num2Bytes(new_value,
                                                       self.length,
                                                       self.signed,
                                                       self.order)    
        
    def incValue(self, buffer, seg_offset, inc_value):
        self.setValue(buffer, 
                      seg_offset,
                      self.getValue(buffer, seg_offset) + inc_value)
        

class BaseTable(DataPiece, Enum):
    
    def __init__(self, *params):  # Python does not have to invoke superclass __init__()
        _len = len(params)
        self.position = params[0]
        self.length = params[1]
        self.end = self.position + self.length
        self.signed = params[2] if _len > 2 else False
        self.order = params[3] if _len > 3 else 'little'
        self.table = params[4] if _len > 4 else None
        self.optimum = params[5] if _len > 5 else None
    

    def getRepr(self, buffer, seg_offset, parent=Tables):
        _value = self.getValue(buffer, seg_offset)
        if self.table:
            try:
                return getattr(parent, self.table)(_value).name
            except ValueError:
                return '0x{0:08X}'.format(_value)               
        else:
            return str(_value)
        
    def optimize(self, buffer, seg_offset):
        if self.optimum is not None:
            self.setValue(buffer, seg_offset, self.optimum)  

In [318]:
class Segments(BaseTable):
    
    Gold = (0x001c, 4, False, 'little', None, 10000)
    States = (0x0020, 4, False, 'little', 'State', None)
    MaxHP = (0x0024, 2, False, 'little', None, 200)
    CurrentHP = (0x0026, 2, False, 'little', None, 200)
    Reputation = (0x0044, 1, True, 'little', None, 20)
    HideInShadow = (0x0045, 1, False, 'little', None, 100)
    NatureAC = (0x0046, 2, True, 'little', None, -10)
    EffectiveAC = (0x0048, 2, True, 'little', None, -10)
    CrushAC = (0x004A, 2, True, 'little', None, -10)
    MissileAC = (0x004C, 2, True, 'little', None, -10)
    PiercingAC = (0x004E, 2, True, 'little', None, -10)
    SlashingAC = (0x0050, 2, True, 'little', None, -10)
    THAC0 = (0x0052, 1, False, 'little', None, 0)
    NumOfAttack = (0x0053, 1, False, 'little', None, 10)
    SaveVsDeath = (0x0054, 1, False, 'little', None, 0)
    SaveVsWands = (0x0055, 1, False, 'little', None, 0)
    SaveVsPolymorph = (0x0056, 1, False, 'little', None, 0)
    SaveVsBreath = (0x0057, 1, False, 'little', None, 0)
    SaveVsSpell = (0x0058, 1, False, 'little', None, 0)
    ResistFire = (0x0059, 1, False, 'little', None, 100)
    ResistCold = (0x005A, 1, False, 'little', None, 100)
    ResistElectricity = (0x005B, 1, False, 'little', None, 100)
    ResistAcid = (0x005C, 1, False, 'little', None, 100)
    ResistMagic = (0x005D, 1, False, 'little', None, 100)
    ResistMagicFire = (0x005E, 1, False, 'little', None, 100)
    ResistMagicCold = (0x005F, 1, False, 'little', None, 100)
    ResistSlashing = (0x0060, 1, False, 'little', None, 100)
    ResistCrushing = (0x0061, 1, False, 'little', None, 100)
    ResistPiercing = (0x0062, 1, False, 'little', None, 100)
    ResistMissile = (0x0063, 1, False, 'little', None, 100)
    DetectIllusion = (0x0064, 1, False, 'little', None, 100)
    SetTraps = (0x0065, 1, False, 'little', None, 100)
    Lore = (0x0066, 1, False,'little',  None, 100)
    Lockpicking = (0x0067, 1, False, 'little', None, 100)
    Stealth = (0x0068, 1,False, 'little',  None, 100)
    FindTraps = (0x0069, 1, False, 'little',None, 100)
    PickPockets = (0x006A, 1, False, 'little', None, 100)
    Fatigue = (0x006B, 1,False, 'little',  None, 0)
    Intoxication = (0x006C, 1, False, 'little', None, 0)
    Luck = (0x006D, 1, False, 'little', None,  100)
    #LSwordPro = (0x006E, 1, False, 'little', None, 5)
    #SSwordPro = (0x006F, 1, False, 'little', None, 5)
    #BowPro = (0x0070, 1, False, 'little', None, 5)
    #SpearPro = (0x0071, 1, False, 'little', None, 5)
    #BluntPro = (0x0072, 1, False, 'little', None, 5)
    #SpikedPro = (0x0073, 1, False, 'little', None, 5)
    #AxePro = (0x0074, 1, False, 'little', None, 5)
    #MissilePro = (0x0075, 1, False, 'little', None, 5)
    #UnusedPro1 = (0x0076, 1, False, 'little', None, 5)
    #UnusedPro2 = (0x0077, 1, False, 'little', None, 5)
    #UnusedPro3 = (0x0078, 1, False, 'little', None, 5)
    #UnusedPro4 = (0x0079, 1, False, 'little', None, 5)
    #UnusedPro5 = (0x007a, 1, False, 'little', None, 5)
    #UnusedPro6 = (0x007b, 1, False, 'little', None, 5)
    #UnusedPro7 = (0x007c, 1, False, 'little', None, 5)
    #NightMareMode = (0x007d, 1, False, 'little', None, None)
    #Translucency = (0x007e, 1, False, 'little', None, None)
    #MurderInc = (0x007f, 1, False, 'little', None, None)
    #ReputationJoin = (0x0080, 1, False, 'little', None, None)
    #ReputationLeave = (0x0081, 1, False, 'little', None, None)
    TurnUndead = (0x0082, 1, False, 'little', None, 100)
    Tracking = (0x0083, 1, False, 'little', None, 100)
    LvClass1 = (0x0234, 1, False, 'little', None, 1)
    LvClass2 = (0x0235, 1, False, 'little', None, 1)
    LvClass3 = (0x0236, 1, False, 'little', None, 1)
    Sex = (0x0237, 1, False, 'little', 'Gender', None)
    Strength = (0x0238, 1, False, 'little', None, 25)
    StrBonus = (0x0239, 1, False, 'little', None, 0)
    Intelligence = (0x023A, 1, False, 'little', None, 25)
    Wisdom = (0x023B, 1, False, 'little', None, 25)
    Dexterity = (0x023C, 1, False, 'little', None, 25)
    Constitution = (0x023D, 1, False, 'little', None, 25)
    Charisma = (0x023E, 1, False, 'little', None, 25)
    Morale = (0x023F, 1, False, 'little', None, 100)
    MoraleBreak = (0x0240, 1, False, 'little', None, 0)
    RacialEnemy = (0x0241, 1, False, 'little', 'Race', None)
    MoraleRecovery = (0x0242, 2, False, 'little', None, 0)
    Kit = (0x0244, 4, False, 'little', 'Kit', None)
    Race = (0x0272, 1, False, 'little', 'Race', None)
    Class = (0x0273, 1, False, 'little', 'Class', None)
    Gender = (0x0275, 1, False, 'little', 'Gender', None)
    Alignment = (0x027B, 1, False, 'little', 'Alignment', None)
    KnownSpellOffset = (0x02a0, 4, False, 'little', None, None)
    KnownSpellCount = (0x02a4, 4, False, 'little', None, None)
    MemSpellOffset = (0x02a8, 4, False, 'little', None, None)
    MemSpellCount = (0x02ac, 4, False, 'little', None, None)
    MemorizedSpellOffset = (0x02b0, 4, False, 'little', None, None)
    MemorizedSpellCount = (0x02b4, 4, False, 'little', None, None)
    ItemSlotOffset = (0x02b8, 4, False, 'little', None, None)
    ItemOffset = (0x02bc, 4, False, 'little', None, None)
    ItemCount = (0x02c0, 4, False, 'little', None, None)
    EffectOffset = (0x02c4, 4, False, 'little', None, None)
    EffectCount = (0x02c8, 4, False, 'little', None, None)
    DialogRef = (0x02cc, 8, False, 'little', None, None)

In [319]:
class BaseSeg(BaseOps):
    
    def __init__(self, filename, name, buffer, parent, extra_internal_offset, offset_ref, length_ref, count_ref):
        self.filename = filename
        self.name = name
        self.buffer = buffer
        self.parent = parent
        self.extra_internal_offset = extra_internal_offset
        self.childrens = list() 
        self.next_sibling = None
        self.offset_ref = offset_ref
        self.length_ref = length_ref
        self.count_ref = count_ref
    
    
    @property
    def _start(self):
        if self.offset_ref:
            return self.offset_ref.getValue(buffer=self.buffer, 
                                            seg_offset=self.parent.adj_offset_buffer)
        else:
            return 0       
        
    @property
    def adj_offset_segment(self):
        return self._start + self.extra_internal_offset

    
    @property
    def adj_offset_buffer(self): 
        if self.parent:
            return self.parent.adj_offset_buffer + self.adj_offset_segment
        else:
            return self.adj_offset_segment      
    
    
    @property
    def length(self):
        if self.length_ref:
            return self.length_ref.getValue(buffer=self.buffer, 
                                            seg_offset=self.parent.adj_offset_buffer)
        else:
            return None
    
    @property
    def count(self):
        if self.count_ref:
            return self.count_ref.getValue(buffer=self.buffer, 
                                           seg_offset=self.parent.adj_offset_buffer)
        else:
            return None
    
    
    def expand(self, volume):
        if self.length_ref:
            self.length_ref.incValue(buffer=self.buffer, seg_offset=self.parent.adj_offset_buffer, inc_value=volume)
        if self.next_sibling:
            self.next_sibling.move(volume)
        if self.parent:
            self.parent.expand(volume)
        
    def move(self, volume):
        self.offset_ref.incValue(buffer=self.buffer, seg_offset=self.parent.adj_offset_buffer, inc_value=volume)
        if self.next_sibling:
            self.next_sibling.move(volume)     
            
    def flush(self):
        with open(self.filename, 'wb+') as _file:
            _file.write(self.buffer)

In [320]:
class CharSeg(BaseSeg):
    
    EXTRA_INTERNAL_OFFSET = 10

    def __init__(self, filename, name, buffer, parent, extra_internal_offset, offset_ref, length_ref):
        super().__init__(filename, name, buffer, parent, extra_internal_offset, offset_ref, length_ref, None)
        self.data = pd.DataFrame()
        self.read()
        self.readAppendix()
        
    def read(self):
        self.data = pd.DataFrame()
        for _name, _item in Segments.__members__.items():
            self.data = self.data.append({'Item': _name,
                                          'File_offset': _item.position + self.adj_offset_buffer,
                                          'CRE_offset': _item.position,
                                          'Length': _item.length,
                                          'Number': _item.getValue(self.buffer, self.adj_offset_buffer),
                                          'Value': _item.getRepr(self.buffer, self.adj_offset_buffer)},
                                        ignore_index=True)
        self.data['File_offset'] = self.data['File_offset'].apply(lambda x: '0x{0:06X}h'.format(int(x)))
        self.data['CRE_offset'] = self.data['CRE_offset'].apply(lambda x: '0x{0:06X}h'.format(int(x)))
        self.data['Length'] = self.data['Length'].astype(int)
        self.data['Number'] = self.data['Number'].apply(lambda x:str(int(x)))
        self.data.set_index('Item', inplace=True)
        self.data.index.name = None
        
    def optimize(self):
        for _name, _item in Segments.__members__.items():
            _item.optimize(self.buffer, self.adj_offset_buffer)
        self.effects.optimize()
        
            
    def setValue(self, item, new_value):
        if not isinstance(new_value, int):
            new_value = new_value.value       
        getattr(Segments, item.name).setValue(self.buffer, self.adj_offset_buffer, new_value)   
        
    def readAppendix(self):
        self.knownSpell = KnownSpell(self.filename, 'KnownSpell', self.buffer, self,
                                     DataPiece(Segments.KnownSpellOffset.position, 
                                               Segments.KnownSpellOffset.length),
                                     DataPiece(Segments.KnownSpellCount.position, 
                                               Segments.KnownSpellCount.length),
                                     12)
        self.memSpell = MemSpell(self.filename, 'MemSpell', self.buffer, self,
                                 DataPiece(Segments.MemSpellOffset.position, 
                                           Segments.MemSpellOffset.length),
                                 DataPiece(Segments.MemSpellCount.position, 
                                           Segments.MemSpellCount.length),
                                 16)
        self.memorizedSpell = MemorizedSpell(self.filename, 'MemorizedSpell', self.buffer, self, 
                                             DataPiece(Segments.MemorizedSpellOffset.position, 
                                                       Segments.MemorizedSpellOffset.length),
                                             DataPiece(Segments.MemorizedSpellCount.position, 
                                                       Segments.MemorizedSpellCount.length),
                                             12)
        self.itemSlots = ItemSlots(self.filename, 'ItemSlots', self.buffer, self,
                                   DataPiece(Segments.ItemSlotOffset.position, 
                                             Segments.ItemSlotOffset.length),
                                   None,
                                   2, 80)
        self.inventoryItems = InventoryItems(self.filename, 'InventoryItems', self.buffer, self, 
                           DataPiece(Segments.ItemOffset.position, 
                                     Segments.ItemOffset.length),
                           DataPiece(Segments.ItemCount.position, 
                                     Segments.ItemCount.length),
                           20)
        self.effects = Effects(self.filename, 'Effects', self.buffer, self, 
                               DataPiece(Segments.EffectOffset.position, 
                                         Segments.EffectOffset.length),
                               DataPiece(Segments.EffectCount.position, 
                                         Segments.EffectCount.length),
                               264)
        self.knownSpell.next_sibling = self.memSpell
        self.memSpell.next_sibling = self.memorizedSpell
        self.memorizedSpell.next_sibling = self.itemSlots
        self.itemSlots.next_sibling = self.inventoryItems
        self.inventoryItems.next_sibling = self.effects
        
    @property
    def weapons(self):
        return self.effects.weapons
    
    @property
    def items(self):
        return self.inventoryItems.items
    
    @property
    def spells(self):
        return self.knownSpell.spells

In [383]:
class AppendixSeg(BaseSeg):
    
    class Record(BaseTable):
        pass
    
    def __init__(self, filename, name, buffer, parent, offset_ref, count_ref, record_length, segment_length=None):
        super().__init__(filename, name, buffer, parent, 0, offset_ref, None, count_ref)
        self.record_length = record_length
        self.data = pd.DataFrame()
        self.records = list()
        self.read()
        
    def read(self):
        self.data = pd.DataFrame()
        self.records = list()
        if self.count:
            for _i in range(self.count):
                _start = self.adj_offset_buffer + _i * self.record_length
                _end = _start + self.record_length
                self.records.append((_start, _end))
                _data = dict([(_name, _item.getRepr(self.buffer, 
                                                    _start)) for _name, _item in self.Record.__members__.items()])
                self.data = self.data.append(_data, ignore_index=True)
                
    def delete(self, index):
        if index < self.count:
            self.buffer[self.records[index][0]: self.records[index][1]] = []
            self.read()
            if self.count:
                self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, -1)
            self.expand(-self.record_length)
    
    def insert(self, template=0):
        if template < self.count:
            self.buffer[self.records[0][0]: self:records[0][0]] = self.buffer[self.records[index][0]: 
                                                                              self:records[index][1]]
            self.read()
            if self.count:
                self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, 1)
            self.expand(self.record_length) 
            
    def setValue(self, item, value, index=0):
        _start = self.records[index][0]
        if not isinstance(new_value, int):
            value = value.value
        getattr(self.Record, item.name).setValue(self.buffer, _start, value)
        
    
    @staticmethod
    def searchName(_value, table):
        _value = _value.replace('\x00', '')
        for _cls in table.__dict__.values():
            if isinstance(_cls, EnumMeta):
                for _item in _cls:
                    if _item.value == _value:
                        return _item.name
        else:
            return _value

In [384]:
class MemSpell(AppendixSeg):
    pass

class MemorizedSpell(AppendixSeg):
    pass

class ItemSlots(AppendixSeg):
    pass

In [385]:
class KnownSpell(AppendixSeg):

    
    class Record(BaseTable):

        SPELL = (0x0000, 8)
        LEVEL = (0x0008, 2)
        TYPE = (0x000a, 2)
        
    SPELL_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    SPELL_PATTERN = re.compile(r'(?P<type>\w{4})(?P<level>\d)(?P<id>\d+)')
    SPELL_TYPE = {'SPWI': 1,
                  'SPCL': 0,
                  'SPIN': 2,
                  'SPPR': 3,
                  1: 'Wizard',
                  0: 'Cleric',
                  2:' Innate',
                  3: 'Special'}
    
    @property
    def end(self):
        return self.adj_offset_buffer + self.count * self.record_length
    
    def read(self):
        self.data = pd.DataFrame(columns=self.Record.__members__.keys())
        self.records = [[], [], [], []]
        for index in range(self.count):
            _start = self.adj_offset_buffer + index * self.record_length
            _end = _start + self.record_length
            _record = self.buffer[_start:_end]
            _spell = self.searchName(self.Record.SPELL.getBytes(_record, 0).decode('UTF-8'),
                                     table=Spells)
            _type = self.Record.TYPE.getValue(_record, 0) 
            _level = self.Record.LEVEL.getValue(_record, 0) + 1
            self.records[_type].append(_record)
            self.data = self.data.append({'SPELL': _spell,
                                          'TYPE': self.SPELL_TYPE[_type],
                                          'LEVEL': _level},
                                        ignore_index=True)
            self.data['LEVEL'] = self.data['LEVEL'].astype(int)
        
        
    
    def prepareTemplate(self, spell):
        _string = re.match(self.SPELL_PATTERN, spell.value)
        _template = self.SPELL_TEMPLATE[:]
        _spell = self.bytes2Num(bytearray(spell.value, encoding='UTF-8'))
        _level = int(_string.group('level')) - 1
        if _level < 0:
            raise('Wait to figure out Lv.10 spells!')
        _type = self.SPELL_TYPE[_string.group('type')]
        self.Record.SPELL.setValue(_template, 0, _spell)
        self.Record.LEVEL.setValue(_template, 0, _level)
        self.Record.TYPE.setValue(_template, 0, _type)
        return _template, _type, _level
    
    
    def _delete(self, _spell):
        _count = 0
        _template, _type, _level = self.prepareTemplate(_spell)
        for _i, _r in self.records[_type]:
            if _r == _spell:
                self.records[_type].pop(_i)
                _count += 1
        return _count
    
    def _insert(self, _spell):
        _count = 0
        _template, _type, _level = self.prepareTemplate(_spell)
        if not _spell in self.records[_type]:
            self.records[_type].append(_template)
            _count += 1
        return _count       
    
    def delete(self, spell):
        self.read()
        _count = 0
        if instance(spell, EnumMeta):
            _count = 0
            for _item, _spell in spell.__members__.items():
                _count += self._delete(_spell)
        else:
            _count = self._delete(spell)
        if _count != 0:
            self.buffer[self.adj_offset_buffer : self.end] = self.merge(self.records)
            self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, -_count)
            self.expand(-self.record_length * _count)
            self.read()
        
    def insert(self, spell):
        _count = 0
        self.read()
        if isinstance(spell, EnumMeta):
            _count = 0
            for _item, _spell in spell.__members__.items():
                _count += self._insert(_spell)
        else:
            _count = self._insert(spell)
        self.buffer[self.adj_offset_buffer : self.end] = self.merge(self.records)
        self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, _count)
        self.expand(self.record_length * _count)
        self.read()
                
    
    def merge(self, spells):
        _array = bytearray()
        for _type in self.records:
            for _record in sorted(_type, key=lambda x: self.Record.LEVEL.getValue(x, 0)):
                _array.extend(_record)
        return _array
    
            
    @property
    def spells(self):
        self.read()
        return self.data

In [386]:
class InventoryItems(AppendixSeg):
    
    class Record(BaseTable):
        NAME = (0x0000, 8)
        EXPIRATION = (0x0008, 1)
        ELAPSED = (0x0009, 1)
        QUANTITY_1 = (0x000a, 2)
        QUANTITY_2 = (0x000c, 2)
        QUANTITY_3 = (0x000e, 2)
        IDENTIFIED = (0x000f, 1)
        UNSTEALABLE = (0x0010, 1)
        STOLEN = (0x0011, 1)
        UNDROPPABLE = (0x0012, 1)
        
    MAX_COUNT = 16
    
    ITEM_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

    
    def read(self):
        self.inventory = list()
        self.records = list()
        self.data = pd.DataFrame(columns=self.Record.__members__.keys())
        for index in range(self.count):
            _start = self.adj_offset_buffer + index * self.record_length
            _end = _start + self.record_length
            _record = self.buffer[_start:_end]
            self.records.append((_start, _end))
            _dict = dict()
            for _item in self.Record:
                if _item == self.Record.NAME:
                    _dict[self.Record.NAME.name] = self.searchName(_item.getBytes(_record, 0).decode('UTF-8'),
                                                                   table=Items)
                else:
                    _dict[_item.name] = _item.getValue(_record, 0)
            self.inventory.append(_dict[self.Record.NAME.name])
            self.data = self.data.append(_dict, ignore_index=True)
        for col in self.data.columns:
            if col != self.Record.NAME.name:
                self.data[col] = self.data[col].astype(int)
                
    def insert(self, item, quality=1):
        self.read()
        if not item.value in self.inventory:
            _template = self.ITEM_TEMPLATE[:]
            self.Record.NAME.setValue(_template, 0, self.bytes2Num(bytearray(item.value, encoding='UTF-8')))
            self.Record.QUANTITY_1.setValue(_template, 0, quality)
            self.buffer[self.records[0][0]:self.records[0][0]] = _template
            self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, 1)
            self.expand(self.record_length)
        self.read()
        
    def delete(self, item):
        self.read()
        for _item, _pos in zip(self.inventory, self.records):
            if item.value == _item:
                self.buffer[_pos[0]: _pos[1]] = []
                self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, -1)
                self.expand(-self.record_length)
        self.read()
        
    @property
    def items(self):
        self.read()
        return self.data

In [387]:
class Effects(AppendixSeg):
    
    class WeaponOptima(Enum):
        BastardSword = 5
        LongSword = 5
        ShortSword = 5
        Axe = 5
        TwoHandedSword = 5
        Katana = 5
        ScimitarWakizashiNinjaTo = 5
        Dagger = 5
        WarHammer = 5
        Spear = 5
        Halberd = 5
        FlailMorningstar = 5
        Mace = 5
        QuarterStaff = 5
        Crossbow = 5
        LongBow = 5
        ShortBow = 5
        Darts = 5
        Sling = 5
        #Blackjack = 5
        #Gun = 5
        #MartialArts = 5
        TwoHandedWeaponSkill = 2
        SwordandShieldSkill = 2
        SingleWeaponSkill = 2
        TwoWeaponSkill = 3
        Club = 5
        #ExtraProficiency2 = 5
        #ExtraProficiency3 = 5
        #ExtraProficiency4 = 5
        #ExtraProficiency5 = 5
        #ExtraProficiency6 = 5
        #ExtraProficiency7 = 5
        #ExtraProficiency8 = 5
        #ExtraProficiency9 = 5
        #ExtraProficiency10 = 5
        #ExtraProficiency11 = 5
        #ExtraProficiency12 = 5
        #ExtraProficiency13 = 5
        #ExtraProficiency14 = 5
        #ExtraProficiency15 = 5
        #ExtraProficiency16 = 5
        #ExtraProficiency17 = 5
        #ExtraProficiency18 = 5
        #ExtraProficiency19 = 5
        #ExtraProficiency20 = 5
    
    WEAPON_OPCODE = 0xe9
    WEAPON_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 233, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 95, 
                                 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 255, 
                                 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    
    class Record(BaseTable):
        OPCODE = (0x0008, 4)
        TARGET = (0x000c, 4, False, 'little', 'Target')
        POWER = (0x0010, 4)
        PARAM_1 = (0x0014, 4, False, 'little', None, 5)
        PARAM_2 = (0x0018, 4, False, 'little', 'WeaponType')
        TIME = (0x001c, 4, False, 'little', 'Time')
        
    def insert(self, template=None):
        if isinstance(template, int) and template < self.count:
            self.buffer[self.records[0][0]: self.records[0][0]] = self.buffer[self.records[index][0]: 
                                                                              self:records[index][1]]
        elif template is None:
            self.buffer[self.records[0][0]: self.records[0][0]] = bytearray(self.WEAPON_TEMPLATE)
        elif isinstance(template, bytearray):
            self.buffer[self.records[0][0]: self.records[0][0]] = template[:]
        self.read()
        if self.count:
            self.count_ref.incValue(self.buffer, self.parent.adj_offset_buffer, 1)
        self.expand(self.record_length) 
        
    @property
    def weapons(self):
        self.read()
        return self.data[self.data['OPCODE'] == str(self.WEAPON_OPCODE)]
    
    def _getWeaponOptimum(self, code):
        _weapon = Tables.WeaponType(code).name
        return getattr(self.WeaponOptima, _weapon).value
    
    def optimize(self, more_weapon=True):
        weapons = list()
        self.read()
        for _start, _end in self.records:
            if self.Record.OPCODE.getValue(self.buffer, _start) == self.WEAPON_OPCODE:
                _code = self.Record.PARAM_2.getValue(self.buffer, _start)
                weapons.append(_code)
                self.Record.PARAM_1.setValue(self.buffer, _start, self._getWeaponOptimum(_code))
        if more_weapon:
            for _weapon, _code in Tables.WeaponType.__members__.items():
                if not _code.value in weapons:
                    _template = self.WEAPON_TEMPLATE[:]
                    self.Record.PARAM_2.setValue(_template, 0, _code)
                    self.Record.PARAM_1.setValue(_template, 0, self._getWeaponOptimum(_code)) 
                    self.insert(_template)
        self.read()

In [388]:
class CRE():
    
    class Blocks(BaseTable):
        NAME = (0x0008, 32)
        CRE_OFFSET = (0x0028, 4)
        CRE_LENGTH = (0x002c, 4)
        
    
    def __init__(self, filename):
        self.filename = filename
        self.pwd = os.path.dirname(self.filename)
        self.file = os.path.basename(self.filename)
        self.zipfile = zipfile.ZipFile(self.filename)
        self.file_list = self.zipfile.namelist()
        self.chr_buffer = bytearray(self.zipfile.read(self.file_list[1]))
        self.adj_offset_buffer = 0
        self.read()
        
    @property
    def name(self):
        _name = self.chr_buffer[self.Blocks.NAME.position:
                                self.Blocks.NAME.position + self.Blocks.NAME.length]
        while _name[-1] == 0:
            _name = _name[:-1]
        return _name.decode('UTF-8')
    
    @property
    def weapons(self):
        return self.cre.weapons
    
    @property
    def spells(self):
        return self.cre.spells
    
    @property
    def items(self):
        return self.cre.items
    
    @property
    def data(self):
        _df = self.cre.data
        return _df[~_df.index.str.contains('Ref|Count|Offset')]

    def expand(self, volume):
        pass
    
    def move(self, volume):
        pass
    
    def read(self):
        self.cre = CharSeg(self.filename,
                           self.name,
                           self.chr_buffer,
                           self,
                           0,
                           DataPiece(self.Blocks.CRE_OFFSET.position,
                                     self.Blocks.CRE_OFFSET.length),
                           DataPiece(self.Blocks.CRE_LENGTH.position,
                                     self.Blocks.CRE_LENGTH.length)
                           )
        
    def optimize(self):
        self.cre.optimize()
        
    def setValue(self, item, new_value):
        self.cre.setValue(item, new_value)
    
    def addSpell(self, spell):
        self.cre.knownSpell.insert(spell)
        
    def removeSpell(self, spell):
        self.cre.knownSpell.delete(spell)
        
    def giveItem(self, item, quantity=1):
        self.cre.inventoryItems.insert(item, quantity)
        
    def removeItem(self, item):
        self.cre.inventoryItems.delete(item)
        
    def pack(self):
        bio_buffer = self.zipfile.read(self.file_list[0])
        self.zipfile.close()
        with zipfile.ZipFile(self.filename, 'w') as target:
            for _f, _name in zip([bio_buffer, self.chr_buffer], self.file_list):
                target.writestr(_name, _f)

In [389]:
filename = r'08BARD.bg1character'
a = CRE(filename)

In [337]:
a.cre.inventoryItems.insert(Items.AMULETS.Agni_Mani_Necklace)
a.removeItem(Items.BAGS.Ammo_Belt)

In [390]:
a.spells

,SPELL,LEVEL,TYPE
0,WIZARD_SPOOK,1,Wizard
1,WIZARD_SLEEP,1,Wizard
2,WIZARD_SHOCKING_GRASP,1,Wizard
3,WIZARD_SHIELD,1,Wizard
4,WIZARD_REFLECTED_IMAGE,1,Wizard
5,WIZARD_PROTECTION_FROM_PETRIFICATION,1,Wizard
6,WIZARD_PROTECTION_FROM_EVIL,1,Wizard
7,WIZARD_ALARM,1,Wizard
8,WIZARD_MAGIC_MISSILE,1,Wizard
9,WIZARD_LARLOCH_MINOR_DRAIN,1,Wizard


In [391]:
a.items

,NAME,EXPIRATION,ELAPSED,QUANTITY_1,QUANTITY_2,QUANTITY_3,IDENTIFIED,UNSTEALABLE,STOLEN,UNDROPPABLE
0,Ammo_Belt,0,0,1,0,0,0,1,0,0
1,Agni_Mani_Necklace,0,0,1,0,0,0,1,0,0
2,Ammo_Belt,0,0,1,0,0,0,1,0,0
3,Agni_Mani_Necklace,0,0,1,0,0,0,1,0,0
4,Ammo_Belt,0,0,1,0,0,0,1,0,0
5,Agni_Mani_Necklace,0,0,1,0,0,0,1,0,0
6,Pale_Green_Ioun_Stone,0,0,0,0,0,0,1,0,0
7,Robe_of_the_Good_Archmagi,0,0,0,0,0,0,1,0,0
8,Katana_PLUS2,0,0,1,0,0,0,1,0,0
9,Amulet_of_Metaspell_Influence,0,0,0,0,0,0,1,0,0


In [317]:
searchName('SPWI106', Spells)

SPWI958
SPWI959
SPWI960
SPCL936
SPCL937
SPCL938
SPIN731
SPIN732
SPIN733
SPIN734
SPIN735
SPIN736
SPIN737
SPIN738
SPIN645
SPIN778
SPIN658
SPIN747
SPIN748
SPIN749
SPIN750
SPIN751
SPIN752
SPIN753
SPIN754
SPIN755
SPIN756
SPIN759
SPIN760
SPIN761
SPIN762
SPIN763
SPIN764
SPIN765
SPIN766
SPIN767
SPIN768
SPIN769
SPIN770
SPIN771
SPIN772
SPIN956
SPCL152
SPPR982
SPPR983
SPPR984
SPPR985
SPPR986
SPPR987
SPPR988
SPPR989
SPWI001
SPWI002
SPWI003
SPWI004
SPWI005
SPWI006
SPWI007
SPWI008
SPWI009
SPWI010
SPWI011
SPWI012
SPWI013
SPWI014
SPWI015
SPWI016
SPWI017
SPWI020
SPWI021
SPWI022
SPWI023
SPWI024
SPWI025
SPWI026
SPWI027
SPIN705
SPIN706
SPIN707
SPIN708
SPIN709
SPIN101
SPIN102
SPIN103
SPIN104
SPIN105
SPIN106
SPCL811
SPCL815
SPCL820
SPIN805
SPCL412
SPCL414
SPCL721
SPCL722
SPCL311
SPCL922
SPIN123
SPCL611
SPCL612
SPCL613
SPCL924
SPCL925
SPPR101
SPPR102
SPPR103
SPPR104
SPPR105
SPPR106
SPPR107
SPPR108
SPPR109
SPPR110
SPPR111
SPPR112
SPPR113
SPPR114
SPPR115
SPPR150
SPPR201
SPPR202
SPPR203
SPPR204
SPPR205
SPPR206


'WIZARD_BLINDNESS'